In [3]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/Appliance-Energy-Prediction")
import pandas as pd
from pca_fts.PcaWeightedMVFTS import PcaWeightedMVFTS
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
import math
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Aux functions

In [4]:
def sample_first_prows(data, perc=0.75):
    return data.head(int(len(data)*(perc)))

In [5]:
# convert series to supervised learning
def series_to_supervised_miso(data, n_in, n_out, endog_var='Appliances', dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(df.columns[j]+'(t-%d)' % (i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df[endog_var].shift(-i))
		if i == 0:
			names += [(endog_var+'(t)')]
		else:
			names += [(endog_var+'(t+%d)' % (i))]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [6]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame) 
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [7]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

## Dataset Smart Home with Weather Information

In [20]:
df = pd.read_csv('/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/data/HomeC.csv')
data = df.drop(labels=['icon','summary','cloudCover', 'House overall [kW]', 'time'], axis=1)
#data = data.loc[0: : 10]
data
data.dropna(inplace=True)
data = clean_dataset(data)
data.head()

/home/hugo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,use [kW],gen [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],Garage door [kW],Kitchen 12 [kW],...,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,windBearing,precipIntensity,dewPoint,precipProbability
0,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
1,0.934333,0.003467,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
2,0.931817,0.003467,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
3,1.022050,0.003483,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
4,1.139400,0.003467,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0


In [21]:
round(data.describe(),3)

,use [kW],gen [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],Garage door [kW],Kitchen 12 [kW],...,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,windBearing,precipIntensity,dewPoint,precipProbability
count,503910.000,503910.000,503910.000,503910.000,503910.000,503910.000,503910.000,503910.000,503910.000,503910.000,...,503910.000,503910.000,503910.000,503910.000,503910.000,503910.000,503910.000,503910.000,503910.000,503910.000
mean,0.859,0.076,0.031,0.099,0.137,0.081,0.064,0.042,0.014,0.003,...,50.742,0.664,9.253,48.263,1016.302,6.650,202.357,0.003,38.694,0.056
std,1.058,0.128,0.191,0.169,0.179,0.104,0.076,0.058,0.014,0.022,...,19.114,0.194,1.611,22.028,7.895,3.983,106.520,0.011,19.088,0.166
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,-12.640,0.130,0.270,-32.080,986.400,0.000,0.000,0.000,-27.240,0.000
25%,0.368,0.003,0.000,0.020,0.064,0.040,0.005,0.007,0.013,0.000,...,35.770,0.510,9.420,31.090,1011.290,3.660,148.000,0.000,24.600,0.000
50%,0.562,0.004,0.000,0.021,0.067,0.042,0.005,0.008,0.013,0.001,...,50.320,0.680,10.000,50.320,1016.530,5.930,208.000,0.000,39.030,0.000
75%,0.970,0.084,0.000,0.069,0.081,0.068,0.125,0.053,0.013,0.001,...,66.260,0.840,10.000,66.260,1021.480,8.940,295.000,0.000,54.790,0.000
max,14.715,0.614,1.402,1.934,0.795,0.972,0.851,1.274,1.089,1.167,...,93.720,0.980,10.000,101.120,1042.460,22.910,359.000,0.191,75.490,0.840


### Dataset Smart Home with Weather Information: Preprocessing, Lag = 1, t+1

In [22]:
data = series_to_supervised_miso(data, 1, 1, endog_var='use [kW]', dropnan=True)
data.head()

,use [kW](t-1),gen [kW](t-1),Dishwasher [kW](t-1),Furnace 1 [kW](t-1),Furnace 2 [kW](t-1),Home office [kW](t-1),Fridge [kW](t-1),Wine cellar [kW](t-1),Garage door [kW](t-1),Kitchen 12 [kW](t-1),...,humidity(t-1),visibility(t-1),apparentTemperature(t-1),pressure(t-1),windSpeed(t-1),windBearing(t-1),precipIntensity(t-1),dewPoint(t-1),precipProbability(t-1),use [kW](t)
1,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,0.934333
2,0.934333,0.003467,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,0.931817
3,0.931817,0.003467,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,1.022050
4,1.022050,0.003483,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,1.139400
5,1.139400,0.003467,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,...,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0,1.391867


In [23]:
round(data.describe(),3)

,use [kW](t-1),gen [kW](t-1),Dishwasher [kW](t-1),Furnace 1 [kW](t-1),Furnace 2 [kW](t-1),Home office [kW](t-1),Fridge [kW](t-1),Wine cellar [kW](t-1),Garage door [kW](t-1),Kitchen 12 [kW](t-1),...,humidity(t-1),visibility(t-1),apparentTemperature(t-1),pressure(t-1),windSpeed(t-1),windBearing(t-1),precipIntensity(t-1),dewPoint(t-1),precipProbability(t-1),use [kW](t)
count,503909.000,503909.000,503909.000,503909.000,503909.000,503909.000,503909.000,503909.000,503909.000,503909.000,...,503909.000,503909.000,503909.000,503909.000,503909.000,503909.000,503909.000,503909.000,503909.000,503909.000
mean,0.859,0.076,0.031,0.099,0.137,0.081,0.064,0.042,0.014,0.003,...,0.664,9.253,48.263,1016.302,6.650,202.357,0.003,38.694,0.056,0.859
std,1.058,0.128,0.191,0.169,0.179,0.104,0.076,0.058,0.014,0.022,...,0.194,1.611,22.028,7.895,3.983,106.521,0.011,19.088,0.166,1.058
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.130,0.270,-32.080,986.400,0.000,0.000,0.000,-27.240,0.000,0.000
25%,0.368,0.003,0.000,0.020,0.064,0.040,0.005,0.007,0.013,0.000,...,0.510,9.420,31.090,1011.290,3.660,148.000,0.000,24.600,0.000,0.368
50%,0.562,0.004,0.000,0.021,0.067,0.042,0.005,0.008,0.013,0.001,...,0.680,10.000,50.320,1016.530,5.930,208.000,0.000,39.030,0.000,0.562
75%,0.970,0.084,0.000,0.069,0.081,0.068,0.125,0.053,0.013,0.001,...,0.840,10.000,66.260,1021.480,8.940,295.000,0.000,54.790,0.000,0.970
max,14.715,0.614,1.402,1.934,0.795,0.972,0.851,1.274,1.089,1.167,...,0.980,10.000,101.120,1042.460,22.910,359.000,0.191,75.490,0.840,14.715


## Model: PcaWeightedMVFTS - PCA-MVFTS

In [24]:
pca_wmvfts = PcaWeightedMVFTS(n_components = 3,
                       endogen_variable = 'use [kW](t-1)',
                       n_part = 50)

In [25]:
result = {
     "window": [],
     "rmse": [],
     "mape": [],
     "smape": [],
     "mae": [],
     "nrmse":[]
}

num_fuzzy_rules = {
    "window": [],
    "qtd": []
}

order = 1
tam = len(data)
n_windows = 30
windows_length = math.floor(tam / n_windows)
for ct, ttrain, ttest in Util.sliding_window(data, windows_length, 0.75, inc=1):
    if len(ttest) > 0:
               
        data_train = ttrain.loc[:,'use [kW](t-1)':'precipProbability(t-1)']
        data_test = ttest.loc[:,'gen [kW](t-1)':'use [kW](t)']
        
        #ALtera o nome da columa porque o modelo utiliza esse label como variável endogena 
        data_test.rename(columns = {'use [kW](t)': 'use [kW](t-1)'}, inplace = True)
        
        print('-' * 20)
        print(f'training window {(ct)}')
        model, pca_reduced_train = pca_wmvfts.run_train_model(data_train)
        forecast, forecast_self, pca_reduced_test = pca_wmvfts.run_test_model(model,data_test)
        
        num_fuzzy_rules["window"].append(ct)
        num_fuzzy_rules["qtd"].append(len(model))
        
        #Teste e cálculo dos erros da previsão  
        #valor use [kW](t-1) é o use [kW](t) --> label igual por causa do modelo      
        print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics")
        rmse = Measures.rmse(pca_reduced_test['use [kW](t-1)'],forecast)
        mape = Measures.mape(pca_reduced_test['use [kW](t-1)'],forecast)
        smape = Measures.smape(pca_reduced_test['use [kW](t-1)'],forecast)
        
        forecast = pd.DataFrame(forecast)
        forecast.fillna(forecast.mean(),inplace=True)
        forecast = np.array(forecast).reshape(-1)
        mae = mean_absolute_error(pca_reduced_test['use [kW](t-1)'], forecast)
        
        nrmse = cal_nrmse(rmse, pca_reduced_test['use [kW](t-1)'])
        
#         # Grafico de comparação Original x Previsto para cada janela
#         fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#         ax.plot(pca_reduced_test['use [kW](t-1)'], label='Original')
#         ax.plot(forecast, label='Forecast')
#         handles, labels = ax.get_legend_handles_labels()
#         lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#         plt.show()
        
        result["rmse"].append(round(rmse,3))
        result["mape"].append(round(mape,3))
        result["smape"].append(round(smape,3))
        result["mae"].append(round(mae,3))
        result["nrmse"].append(round(nrmse,3))
        result["window"].append(ct)
        
measures = pd.DataFrame(result)
rules = pd.DataFrame(num_fuzzy_rules)



/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


--------------------
training window 0
[ 23:37:25] getting statistics
--------------------
training window 16796


/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


[ 23:38:07] getting statistics
--------------------
training window 33592


/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


...

[ 23:55:51] getting statistics
--------------------
training window 470288


/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


[ 23:56:35] getting statistics
--------------------
training window 487084


/home/hugo/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


[ 23:57:14] getting statistics


In [27]:
measures

,window,rmse,mape,smape,mae,nrmse
0,0,0.018,0.674,0.320,0.004,0.004
1,16796,0.022,0.761,0.303,0.006,0.003
2,33592,0.020,0.406,0.199,0.003,0.005
3,50388,0.031,0.482,0.247,0.006,0.005
4,67184,0.040,1.551,0.757,0.013,0.006
5,83980,0.041,2.999,1.352,0.013,0.011
6,100776,0.014,0.304,0.148,0.002,0.002
7,117572,0.012,0.281,0.136,0.002,0.002
8,134368,0.013,0.729,0.358,0.004,0.003
9,151164,0.006,0.511,0.219,0.012,0.001


In [17]:
#measures.to_csv (r'win_pca_wmvfts_ksh_10min.csv', index = False, header=True)

In [28]:
measures.to_csv (r'win_pca_wmvfts_ksh_1min.csv', index = False, header=True)

### Mean Statistics PCA-WMVFTS:

In [30]:
final_result = {
     "rmse": [],
     "mape": [],
     "smape": [],
     "mae": [],
     "nrmse":[],
     "rmse_std": [],
     "mae_std": [],
     "mape_std": [],
     "smape_std": [],
     "nrmse_std": [],
     "rules":[]
}

final_result["rmse"].append(round(statistics.mean(measures['rmse']),3))
final_result["mape"].append(round(statistics.mean(measures['mape']),3))
final_result["smape"].append(round(statistics.mean(measures['smape']),3))
final_result["mae"].append(round(statistics.mean(measures['mae']),3))
final_result["nrmse"].append(round(statistics.mean(measures['nrmse']),3))

final_result["rmse_std"].append(round(statistics.stdev(measures['rmse']),3))
final_result["mape_std"].append(round(statistics.stdev(measures['mape']),3))
final_result["smape_std"].append(round(statistics.stdev(measures['smape']),3))
final_result["mae_std"].append(round(statistics.stdev(measures['mae']),3))
final_result["nrmse_std"].append(round(statistics.stdev(measures['nrmse']),3))

final_result["rules"].append(round(statistics.mean(rules.loc[:,'qtd']),3))

final_measures_pca = pd.DataFrame(final_result)

print("Mean Statistics PCA-WMVFTS (test): ")
final_measures_pca

Mean Statistics PCA-WMVFTS (test): 


,rmse,mape,smape,mae,nrmse,rmse_std,mae_std,mape_std,smape_std,nrmse_std,rules
0,0.024,0.956,0.41,0.005,0.004,0.015,0.003,0.782,0.307,0.003,29933.1


In [19]:
#final_measures_pca.to_csv (r'pca_wmvfts_ksh_10min.csv', index = False, header=True)

In [31]:
final_measures_pca.to_csv (r'pca_wmvfts_ksh_1min.csv', index = False, header=True)

### Extras 

In [47]:
# pca_reduced_train

In [46]:
# pca_reduced_test

In [ ]:
# print(model)

In [ ]:
# train = sample_first_prows(data,0.75)
# test = data.iloc[max(train.index):]
# y_test = data.iloc[max(train.index):]['Appliances_t+1'].values

In [ ]:
# pca_wmvfts = PcaWeightedMVFTS(n_components = 2,
#                        endogen_variable = 'Appliances_t+1',
#                        n_part = 50)

In [ ]:
# model, pca_reduced_train = pca_wmvfts.run_train_model(train)
# forecast, forecast_self, pca_reduced_test = pca_wmvfts.run_test_model(model,test)

In [ ]:
#model.mvflrg

In [ ]:
# forecast_1 = model.predict(pca_reduced_test)

In [ ]:
# Measures.rmse(y_test,forecast)

In [ ]:
# Measures.rmse(forecast_1,y_test)

In [ ]:
# Measures.rmse(y_test,forecast_self)

In [ ]:
# fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
# ax.plot(y_test, label='Original')
# ax.plot(forecast, label='Forecast')
# handles, labels = ax.get_legend_handles_labels()
# lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
# plt.show()


In [ ]:
# diff = data.loc[:,'lights':'Tdewpoint']
# diff = diff.diff()
# diff['Appliances'] = data['Appliances']
# data = diff.dropna()